In [33]:
from ipython2cwl.iotypes import CWLFilePathInput, CWLStringInput, CWLFilePathOutput

In [35]:
# http://chromedriver.chromium.org/downloads を参考にインストールされるgoogle chromeのバージョンに対応するdriverのバージョンに適宜修正してください
driver_version: CWLStringInput = "86.0.4240.22"
!pip install chromedriver-binary~=$driver_version

  Created wheel for chromedriver-binary: filename=chromedriver_binary-86.0.4240.22.0-py3-none-any.whl size=7804685 sha256=ad8148ce451d7d609e1211f5245fc8737ee0867bea4b62918c372420922ed243
  Stored in directory: /Users/kawashima/Library/Caches/pip/wheels/82/87/41/90a8da305176a7cf8bfa4be0787159c7405f59fbe1dbad3c8b
Successfully built chromedriver-binary
  Attempting uninstall: chromedriver-binary
    Found existing installation: chromedriver-binary 87.0.4280.20.0
    Uninstalling chromedriver-binary-87.0.4280.20.0:
      Successfully uninstalled chromedriver-binary-87.0.4280.20.0


In [37]:
import os
import datetime
from enum import Enum

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import chromedriver_binary

In [38]:
driver = webdriver.Chrome()

In [39]:
# 保存ディレクトリ(デフォルトは日付)
outdir: CWLFilePathInput = datetime.date.strftime(datetime.date.today(), "%Y%m%d")
outdir

'20201019'

In [40]:
url: CWLStringInput = "https://pdbj.org/featured/covid-19"
driver.get(url)

In [42]:
class Page(Enum):
    ALL = 1
    REPR = 2
    LATEST = 3

def move_page(driver, target: Page):
    if target == Page.ALL:
        element = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/div[2]/span[1]')
    if target == Page.REPR:
        element = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/div[2]/span[2]')
    if target == Page.LATEST:
        element = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/div[2]/span[3]')
    loc = element.location.values()
    actions = ActionChains(driver)
    element.location_once_scrolled_into_view
    actions.move_to_element(element)
    actions.click()
    actions.perform()

In [44]:
os.makedirs( os.path.join(outdir, "all"), exist_ok=True)

i = 1

move_page(driver, Page.ALL)
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div')))
while True:
    html = driver.page_source
    with open(os.path.join(outdir, "all", str(i) + ".html"), "w") as f:
        f.write(html)
    i += 1
    try:
        driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/div[3]/div/div/a[text() = ">"]').click()
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[@class = "loadingImage" and @class != "visible"]')))
    except:
        break

move_page(driver, Page.LATEST)
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div'))) 
while True:
    html = driver.page_source
    with open(os.path.join(outdir, "all", str(i) + ".html"), "w") as f:
        f.write(html)
    i += 1
    try:
        driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/div[3]/div/div/a[text() = ">"]').click()
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[@class = "loadingImage" and @class != "visible"]')))
    except:
        break


In [45]:
os.makedirs(os.path.join(outdir, "rep"), exist_ok=True)

i = 1

move_page(driver, Page.REPR)
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div')))
while True:
    html = driver.page_source
    with open(os.path.join(outdir, "rep", str(i) + ".html"), "w") as f:
        f.write(html)
    i += 1
    try:
        driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/div[3]/div/div/a[text() = ">"]').click()
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[@class = "loadingImage" and @class != "visible"]')))
    except:
        break